In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from ipywidgets import widgets, interact, interactive, HBox, VBox
from sklearn.preprocessing import StandardScaler
from sklearn.cluster import KMeans
import plotly.express as px
from bokeh.plotting import figure, show
from bokeh.io import output_notebook
output_notebook()

df = pd.read_csv("transactions.csv")
df["Date"] = pd.to_datetime(df["Date"])

current_date = df['Date'].max() + pd.Timedelta(days=1)
rfm = df.groupby("Customer_Id").agg({
    "Date": lambda x: (current_date - x.max()).days,
    "Transaction_Id": "nunique",
    "Amount": "sum"
}).reset_index()
rfm.columns = ["Customer_Id", "Recency", "Frequency", "Monetary"]

sc = StandardScaler()
rfm_scaled = sc.fit_transform(rfm[["Recency","Frequency","Monetary"]])
kmeans = KMeans(n_clusters=3, random_state=42, n_init=10)
rfm["Cluster"] = kmeans.fit_predict(rfm_scaled)

graph_selector = widgets.SelectMultiple(
    options=[
        "Top Items (Plotly)",
        "RFM Distribution (Seaborn)",
        "Cluster Plot (Plotly)",
        "Top Items (Bokeh)"
    ],
    value=["Top Items (Plotly)"],
    description="Graphs",
    rows=6,
    style={'description_width': 'initial'}
)

rfm_dropdown = widgets.Dropdown(
    options=["Recency", "Frequency", "Monetary"],
    value="Recency",
    description="RFM Feature:",
)

top_n_slider = widgets.IntSlider(
    value=10,
    min=5,
    max=20,
    step=1,
    description="Top N Items:",
    continuous_update=False
)

button = widgets.Button(
    description="Generate Dashboard",
    button_style="success",
    layout={'width': '200px'}
)

output = widgets.Output()

def generate_dashboard(b):
    with output:
        output.clear_output()
        
        selections = graph_selector.value
        
        if "Top Items (Plotly)" in selections:
            item_counts = df["Item"].value_counts().head(top_n_slider.value)
            fig = px.bar(
                item_counts,
                x=item_counts.values,
                y=item_counts.index,
                orientation="h",
                title=f"Top {top_n_slider.value} Purchased Items (Plotly)",
                text=item_counts.values
            )
            fig.show()

        if "RFM Distribution (Seaborn)" in selections:
            plt.figure(figsize=(10,5))
            sns.histplot(rfm[rfm_dropdown.value], bins=20, kde=True, color="purple")
            plt.title(f"{rfm_dropdown.value} Distribution", fontsize=16)
            plt.show()

        if "Cluster Plot (Plotly)" in selections:
            fig = px.scatter(
                rfm,
                x="Frequency",
                y="Monetary",
                color="Cluster",
                title="Customer Cluster Scatter Plot",
                size_max=12
            )
            fig.show()

        if "Top Items (Bokeh)" in selections:
            item_counts = df["Item"].value_counts().head(top_n_slider.value).reset_index()
            item_counts.columns = ["Item", "Count"]

            p = figure(
                y_range=item_counts["Item"][::-1],
                height=400,
                title=f"Top {top_n_slider.value} Items (Bokeh)"
            )
            p.hbar(y=item_counts["Item"], right=item_counts["Count"], height=0.5, color="orange")
            show(p)

button.on_click(generate_dashboard)

display(
    VBox([
        widgets.HTML("<h2 style='color:green'>🛒 Interactive Customer Analytics Dashboard</h2>"),
        graph_selector,
        rfm_dropdown,
        top_n_slider,
        button,
        output
    ])
)

Loading BokehJS ...